### Analysis of Liverpool 2023 Voting
This analysis assumes different voting schemes, which all have in common that all countries apart from the last get points. This scheme would work for 25 participants, too, as then the last act would get 1 point instead of 0.

First case assumes that the points from 0 - 26 are distributed evenly, while the second scheme assumes that the second best gets 26 and the best 28 points.

The data is scraped from eurovision.tv for all countries

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
base_path = 'https://eurovision.tv/event/liverpool-2023/grand-final/results/{0}'
header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
# countries = ('albania', 'armenia', 'australia', 'austria', 'belgium', 'croatia', 'cyprus', 'czechia', 'estonia', 'finland', 'france', 'germany', 'israel', 'italy', 'lithuania', 'moldova', 'norway', 'poland', 'portugal', 'serbia', 'slovenia', 'spain', 'sweden', 'switzerland', 'ukraine', 'united kingdom')
voting_type = ('juryA', 'juryB', 'juryC', 'juryD', 'juryE')
load_from_internet = True

Reading out the names of the voting countries from the list of countries entry

In [3]:

respones_for_countries = requests.get(base_path.format('albania'), headers=header)
countries_soup = BeautifulSoup(respones_for_countries.text, 'html.parser')


In [4]:
countries = []
country_selection = countries_soup.find('select', class_='form-select')
for con in country_selection.children:
    countries.append(con.text.lower().replace(' ', '-'))

First, lets create the final data structure to be populated. The data structure contains a line for each country, and the columns are the origins of the points.
The origins are named by the following scheme: '<country>_tele', '<country>_juryA', etc

In [5]:
columns_jury = []
columns_jury_final = []
columns_tele = []
for voted_to_coun in countries:
    for vote in voting_type:
        columns_jury.append(f'{voted_to_coun}_{vote}')
    columns_tele.append(f'{voted_to_coun}_tele')
    columns_jury_final.append(f'{voted_to_coun}_jury')
        
votes_jury = pd.DataFrame(index=countries, columns=columns_jury)
votes_tele = pd.DataFrame(index=countries, columns=columns_tele)

In [6]:
if load_from_internet:
    for voted_to_coun in countries:
        voted_to_coun = voted_to_coun.replace(' ', '-')
        print(base_path.format(voted_to_coun))
        response = requests.get(base_path.format(voted_to_coun), headers=header)
        soup = BeautifulSoup(response.text, 'html.parser')
        detailed_voting = soup.find('table').find('tbody')
        rows = detailed_voting.findAll('tr')
        for row in rows:
            cols = row.findAll('td')
            origin = cols[0].text.strip().lower().replace(' ', '-')
            # Add jury votes
            votes_jury.loc[origin, f'{voted_to_coun}_juryA'] = int(cols[2].text.strip())
            votes_jury.loc[origin, f'{voted_to_coun}_juryB'] = int(cols[3].text.strip())
            votes_jury.loc[origin, f'{voted_to_coun}_juryC'] = int(cols[4].text.strip())
            votes_jury.loc[origin, f'{voted_to_coun}_juryD'] = int(cols[5].text.strip())
            if len(cols) == 9:
                votes_jury.loc[origin, f'{voted_to_coun}_juryE'] = int(cols[6].text.strip())
            # Add televotes (more complicated due to structure)
            votes_tele.loc[origin, f'{voted_to_coun}_tele'] = int(cols[-1].text.split(' ')[-1].strip('stndrh'))
else:
    votes_jury = pd.read_csv(r'voting_results_table_jury.csv')
    votes_tele = pd.read_csv(r'voting_results_table_tele.csv')

https://eurovision.tv/event/liverpool-2023/grand-final/results/albania
https://eurovision.tv/event/liverpool-2023/grand-final/results/armenia
https://eurovision.tv/event/liverpool-2023/grand-final/results/australia
https://eurovision.tv/event/liverpool-2023/grand-final/results/austria
https://eurovision.tv/event/liverpool-2023/grand-final/results/azerbaijan
https://eurovision.tv/event/liverpool-2023/grand-final/results/belgium
https://eurovision.tv/event/liverpool-2023/grand-final/results/croatia
https://eurovision.tv/event/liverpool-2023/grand-final/results/cyprus
https://eurovision.tv/event/liverpool-2023/grand-final/results/czechia
https://eurovision.tv/event/liverpool-2023/grand-final/results/denmark
https://eurovision.tv/event/liverpool-2023/grand-final/results/estonia
https://eurovision.tv/event/liverpool-2023/grand-final/results/finland
https://eurovision.tv/event/liverpool-2023/grand-final/results/france
https://eurovision.tv/event/liverpool-2023/grand-final/results/georgia
htt

In [7]:
# save data frame to file to not depend on internet connection
votes_jury.to_csv(r'voting_results_table_jury.csv')
votes_tele.to_csv(r'voting_results_table_tele.csv')

Combining jury votes per country, ordering them from lowest to highest rank, and reassigning 1st to 26th place (26th place will always be the country itself getting 0 points from itself)

### Implement voting schema

#### First trying to reproduce the official voting scheme

This is not a piece of cake, as the voting system is maximally intransparent... See e.g. https://thateurovisionsite.com/2023/04/12/eurovision-2023-voting-rules/

Each jury member ranks from 1-25, this is converted to points from 12 decreasing exponentially. These points from all judges are combined and then sorted again, and the first 10 get the usual points.

In [8]:
votes_jury_combined = pd.DataFrame(index=countries, columns=columns_jury_final)

Assuming that "decreasing exponentially" is ment literally as a function 

In [9]:
jury_points = tuple(12*np.exp(-pos) for pos in range(25))

In [13]:
votes_jury.rank(axis=0)

,albania_juryA,albania_juryB,albania_juryC,albania_juryD,albania_juryE,armenia_juryA,armenia_juryB,armenia_juryC,armenia_juryD,armenia_juryE,...,ukraine_juryA,ukraine_juryB,ukraine_juryC,ukraine_juryD,ukraine_juryE,united-kingdom_juryA,united-kingdom_juryB,united-kingdom_juryC,united-kingdom_juryD,united-kingdom_juryE
albania,NaN,NaN,NaN,NaN,NaN,30.5,18.0,1.0,4.0,2.0,...,25.5,32.0,20.5,27.0,4.5,3.0,15.5,2.0,4.5,29.5
armenia,35.0,33.5,35.0,32.0,31.5,NaN,NaN,NaN,NaN,NaN,...,19.5,4.5,14.5,18.0,4.5,20.0,32.5,3.0,2.5,13.0
australia,10.0,4.5,10.5,11.0,19.5,25.0,25.5,20.5,33.0,22.0,...,25.5,32.0,29.0,29.5,6.0,4.5,27.0,20.0,12.5,22.5
austria,21.0,33.5,12.0,6.0,27.0,25.0,11.0,4.0,25.5,17.0,...,7.0,13.5,23.5,19.0,7.0,31.5,18.5,27.5,34.0,14.5
azerbaijan,1.0,12.5,1.0,1.0,6.0,35.0,28.5,35.0,36.0,30.0,...,8.5,17.5,4.5,8.5,20.5,13.0,5.5,30.5,2.5,29.5
belgium,21.0,12.5,8.0,15.5,14.5,32.0,8.0,28.5,22.0,33.5,...,25.5,36.0,34.0,35.5,17.0,16.5,22.5,15.0,14.5,14.5
croatia,14.5,4.5,14.0,25.0,3.0,36.0,33.5,26.0,28.5,22.0,...,34.0,29.0,31.5,5.0,28.5,10.5,8.0,15.0,20.0,17.0
cyprus,30.5,20.0,31.5,7.5,6.0,25.0,15.5,23.5,1.5,8.0,...,29.0,10.0,7.5,31.5,24.0,23.0,8.0,27.5,34.0,9.0
czechia,2.0,22.5,24.5,13.0,6.0,2.0,4.0,23.5,5.0,1.0,...,8.5,1.0,1.5,1.0,8.5,31.5,30.0,33.0,18.0,18.0
denmark,25.5,28.0,35.0,35.5,11.0,18.0,28.5,23.5,13.0,18.5,...,14.0,20.5,36.0,15.0,33.5,6.5,13.0,1.0,6.5,7.0


In [10]:
for voting_coun in countries:
    for voted_to_coun in countries:
        votes_jury_combined.loc[voting_coun, f'{voted_to_coun}_jury'] = votes_jury.loc[voting_coun, (col for col in votes_jury if col.startswith(f'{voted_to_coun}'))].rank(axis=0)

ValueError: Incompatible indexer with Series

Convert the average placing into a placing from 1st to last

In [ ]:
votes_jury_ranking = pd.DataFrame(0, index=countries, columns=columns_jury_final)

for voting_coun in countries:
    votes_jury_ranking.loc[voting_coun, :] = votes_jury_combined.loc[voting_coun, :].rank()